<a href="https://colab.research.google.com/github/johncoogan53/HW4/blob/main/hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/My Drive/DLHW4')

### Lab2 (a) Model preperation

In [4]:
from resnet20 import ResNetCIFAR
from train_util import train, finetune, test
import torch
import numpy as np

import time

import torchvision.transforms as transforms
import torchvision
import torch.nn as nn
import torch.optim as optim

from FP_layers import *

device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
print(device)

cuda


In [5]:
net = ResNetCIFAR(num_layers=20, Nbits=None)
net = net.to(device)

In [10]:
# Load the best weight paramters
net.load_state_dict(torch.load("/content/drive/My Drive/DLHW4/pretrained_model.pt"))
test(net)

<ipython-input-10-7a50f8b27ae8>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("/content/drive/My Drive/DLHW4/pretrained_model.pt"))


100%|██████████| 170M/170M [00:04<00:00, 35.4MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Test Loss=0.3231, Test accuracy=0.9151


### Lab2 (b) Prune by percentage

In [11]:
def prune_by_percentage(layer, q=70.0):
    """
    Pruning the weight paramters by threshold.
    :param q: pruning percentile. 'q' percent of the least
    significant weight parameters will be pruned.
    """
    # Convert the weight of "layer" to numpy array
    weights = layer.weight.data.cpu().numpy()
    # Compute the q-th percentile of the abs of the converted array
    threshold = np.percentile(np.abs(weights), q)
    # Generate a binary mask same shape as weight to decide which element to prune
    mask = np.abs(weights) >= threshold
    # Convert mask to torch tensor and put on GPU
    mask_tensor = torch.tensor(mask, dtype=torch.float32, device=layer.weight.device)
    # Multiply the weight by mask to perform pruning
    layer.weight.data.mul_(mask_tensor)

    pass

In [26]:
net.load_state_dict(torch.load("/content/drive/My Drive/DLHW4/pretrained_model.pt"))

for name, layer in net.named_modules():
    if (
        isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)
    ) and "id_mapping" not in name:
        # change q value
        prune_by_percentage(layer, q=70.0)

        # Optional: Check the sparsity you achieve in each layer
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.data.cpu().numpy()
        # Count number of zeros
        zeros = np.sum(np_weight == 0)
        # Count number of parameters
        total = np_weight.size
        # Print sparsity
        print("Sparsity of " + name + ": " + str(zeros / total))

test(net)

<ipython-input-26-2fe05e8c2c61>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("/content/drive/My Drive/DLHW4/pretrained_model.pt"))


Sparsity of head_conv.0.conv: 0.6990740740740741
Sparsity of body_op.0.conv1.0.conv: 0.7000868055555556
Sparsity of body_op.0.conv2.0.conv: 0.7000868055555556
Sparsity of body_op.1.conv1.0.conv: 0.7000868055555556
Sparsity of body_op.1.conv2.0.conv: 0.7000868055555556
Sparsity of body_op.2.conv1.0.conv: 0.7000868055555556
Sparsity of body_op.2.conv2.0.conv: 0.7000868055555556
Sparsity of body_op.3.conv1.0.conv: 0.6998697916666666
Sparsity of body_op.3.conv2.0.conv: 0.6999782986111112
Sparsity of body_op.4.conv1.0.conv: 0.6999782986111112
Sparsity of body_op.4.conv2.0.conv: 0.6999782986111112
Sparsity of body_op.5.conv1.0.conv: 0.6999782986111112
Sparsity of body_op.5.conv2.0.conv: 0.6999782986111112
Sparsity of body_op.6.conv1.0.conv: 0.6999782986111112
Sparsity of body_op.6.conv2.0.conv: 0.7000054253472222
Sparsity of body_op.7.conv1.0.conv: 0.7000054253472222
Sparsity of body_op.7.conv2.0.conv: 0.7000054253472222
Sparsity of body_op.8.conv1.0.conv: 0.7000054253472222
Sparsity of body

### Lab2 (c) Finetune pruned model

In [17]:
def finetune_after_prune(net, trainloader, criterion, optimizer, prune=True):
    """
    Finetune the pruned model for a single epoch
    Make sure pruned weights are kept as zero
    """
    # Build a dictionary for the nonzero weights
    weight_mask = {}
    for name, layer in net.named_modules():
        if (
            isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)
        ) and "id_mapping" not in name:
            # Your code here: generate a mask in GPU torch tensor to have 1 for nonzero element and 0 for zero element
            weight_mask[name] = (layer.weight.detach().cpu().numpy() != 0).astype(float)
            weight_mask[name] = torch.tensor(
                weight_mask[name], dtype=torch.float32, device=layer.weight.device
            )

    global_steps = 0
    train_loss = 0
    correct = 0
    total = 0
    start = time.time()
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        if prune:
            for name, layer in net.named_modules():
                if (
                    isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)
                ) and "id_mapping" not in name:
                    # Your code here: Use weight_mask to make sure zero elements remains zero
                    layer.weight.data.mul_(weight_mask[name])

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        global_steps += 1

        if global_steps % 50 == 0:
            end = time.time()
            batch_size = 256
            num_examples_per_second = 50 * batch_size / (end - start)
            print(
                "[Step=%d]\tLoss=%.4f\tacc=%.4f\t%.1f examples/second"
                % (
                    global_steps,
                    train_loss / (batch_idx + 1),
                    (correct / total),
                    num_examples_per_second,
                )
            )
            start = time.time()

In [19]:
# Get pruned model
net.load_state_dict(torch.load("/content/drive/My Drive/DLHW4/pretrained_model.pt"))
for name, layer in net.named_modules():
    if (
        isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)
    ) and "id_mapping" not in name:
        prune_by_percentage(layer, q=70.0)

# Training setup, do not change
batch_size = 256
lr = 0.002
reg = 1e-4

print("==> Preparing data..")
transform_train = transforms.Compose(
    [
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)

transform_test = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
trainset = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform_train
)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=batch_size, shuffle=True, num_workers=16
)

testset = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform_test
)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(
    net.parameters(), lr=lr, momentum=0.875, weight_decay=reg, nesterov=False
)

<ipython-input-19-29d5f8d23722>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("/content/drive/My Drive/DLHW4/pretrained_model.pt"))


==> Preparing data..
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Files already downloaded and verified


In [20]:
# Model finetuning
for epoch in range(20):
    print("\nEpoch: %d" % epoch)
    net.train()
    finetune_after_prune(net, trainloader, criterion, optimizer)
    # Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))

    if val_acc > best_acc:
        best_acc = val_acc
        print("Saving...")
        torch.save(net.state_dict(), "/content/drive/My Drive/DLHW4/net_after_finetune.pt")


Epoch: 0
[Step=50]	Loss=0.3978	acc=0.8627	1298.8 examples/second
[Step=100]	Loss=0.3551	acc=0.8773	2827.1 examples/second
[Step=150]	Loss=0.3323	acc=0.8857	2547.1 examples/second
Test Loss=0.4261, Test acc=0.8667
Saving...

Epoch: 1
[Step=50]	Loss=0.2491	acc=0.9147	1636.9 examples/second
[Step=100]	Loss=0.2410	acc=0.9179	1737.4 examples/second
[Step=150]	Loss=0.2392	acc=0.9179	2797.5 examples/second
Test Loss=0.3938, Test acc=0.8754
Saving...

Epoch: 2
[Step=50]	Loss=0.2200	acc=0.9226	1749.7 examples/second
[Step=100]	Loss=0.2159	acc=0.9236	2234.5 examples/second
[Step=150]	Loss=0.2139	acc=0.9244	2138.6 examples/second
Test Loss=0.3795, Test acc=0.8794
Saving...

Epoch: 3
[Step=50]	Loss=0.2008	acc=0.9305	1330.7 examples/second
[Step=100]	Loss=0.2011	acc=0.9300	2757.1 examples/second
[Step=150]	Loss=0.1994	acc=0.9311	2349.1 examples/second
Test Loss=0.3707, Test acc=0.8826
Saving...

Epoch: 4
[Step=50]	Loss=0.1950	acc=0.9304	1708.7 examples/second
[Step=100]	Loss=0.1871	acc=0.9330	1945

In [22]:
# Check sparsity of the finetuned model, make sure it's not changed
net.load_state_dict(torch.load("/content/drive/My Drive/DLHW4/net_after_finetune.pt"))

for name, layer in net.named_modules():
    if (
        isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)
    ) and "id_mapping" not in name:
        # Your code here:
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.data.cpu().numpy()
        # Count number of zeros
        zeros = np.sum(np_weight == 0)
        # Count number of parameters
        total = np_weight.size
        # Print sparsity
        print("Sparsity of " + name + ": " + str(zeros / total))

test(net)

<ipython-input-22-2b35f04f0af1>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("/content/drive/My Drive/DLHW4/net_after_finetune.pt"))


Sparsity of head_conv.0.conv: 0.6990740740740741
Sparsity of body_op.0.conv1.0.conv: 0.7000868055555556
Sparsity of body_op.0.conv2.0.conv: 0.7000868055555556
Sparsity of body_op.1.conv1.0.conv: 0.7000868055555556
Sparsity of body_op.1.conv2.0.conv: 0.7000868055555556
Sparsity of body_op.2.conv1.0.conv: 0.7000868055555556
Sparsity of body_op.2.conv2.0.conv: 0.7000868055555556
Sparsity of body_op.3.conv1.0.conv: 0.6998697916666666
Sparsity of body_op.3.conv2.0.conv: 0.6999782986111112
Sparsity of body_op.4.conv1.0.conv: 0.6999782986111112
Sparsity of body_op.4.conv2.0.conv: 0.6999782986111112
Sparsity of body_op.5.conv1.0.conv: 0.6999782986111112
Sparsity of body_op.5.conv2.0.conv: 0.6999782986111112
Sparsity of body_op.6.conv1.0.conv: 0.6999782986111112
Sparsity of body_op.6.conv2.0.conv: 0.7000054253472222
Sparsity of body_op.7.conv1.0.conv: 0.7000054253472222
Sparsity of body_op.7.conv2.0.conv: 0.7000054253472222
Sparsity of body_op.8.conv1.0.conv: 0.7000054253472222
Sparsity of body

### Lab2 (d) Iterative pruning

In [12]:
net.load_state_dict(torch.load("/content/drive/My Drive/DLHW4/pretrained_model.pt"))
best_acc = 0.0
for epoch in range(20):
    print("\nEpoch: %d" % epoch)

    net.train()
    if epoch < 10:
        for name, layer in net.named_modules():
            if (
                isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)
            ) and "id_mapping" not in name:
                # Increase model sparsity
                q = (epoch + 1) * 7
                prune_by_percentage(layer, q=q)
    if epoch < 9:
        finetune_after_prune(net, trainloader, criterion, optimizer, prune=False)
    else:
        finetune_after_prune(net, trainloader, criterion, optimizer)

    # Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))

    if epoch >= 10:
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving...")
            torch.save(net.state_dict(), "/content/drive/My Drive/DLHW4/net_after_iterative_prune.pt")


Epoch: 0


<ipython-input-12-f991dd140570>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("/content/drive/My Drive/DLHW4/pretrained_model.pt"))


NameError: name 'prune_by_percentage' is not defined

In [13]:
# Check sparsity of the final model, make sure it's 70%
net.load_state_dict(torch.load("/content/drive/My Drive/DLHW4/net_after_iterative_prune.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.data.cpu().numpy()
        # Count number of zeros
        zeros = np.sum(np_weight == 0)
        # Count number of parameters
        total = np_weight.size
        # Print sparsity
        print("Sparsity of " + name + ": " + str(zeros / total))
#updates
test(net)

<ipython-input-13-928cbabe87b8>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("/content/drive/My Drive/DLHW4/net_after_iterative_prune.pt")

Sparsity of head_conv.0.conv: 0.6990740740740741
Sparsity of body_op.0.conv1.0.conv: 0.7000868055555556
Sparsity of body_op.0.conv2.0.conv: 0.7000868055555556
Sparsity of body_op.1.conv1.0.conv: 0.7000868055555556
Sparsity of body_op.1.conv2.0.conv: 0.7000868055555556
Sparsity of body_op.2.conv1.0.conv: 0.7000868055555556
Sparsity of body_op.2.conv2.0.conv: 0.7000868055555556
Sparsity of body_op.3.conv1.0.conv: 0.6998697916666666
Sparsity of body_op.3.conv2.0.conv: 0.6999782986111112
Sparsity of body_op.4.conv1.0.conv: 0.6999782986111112
Sparsity of body_op.4.conv2.0.conv: 0.6999782986111112
Sparsity of body_op.5.conv1.0.conv: 0.6999782986111112
Sparsity of body_op.5.conv2.0.conv: 0.6999782986111112
Sparsity of body_op.6.conv1.0.conv: 0.6999782986111112
Sparsity of body_op.6.conv2.0.conv: 0.7000054253472222
Sparsity of body_op.7.conv1.0.conv: 0.7000054253472222
Sparsity of body_op.7.conv2.0.conv: 0.7000054253472222
Sparsity of body_op.8.conv1.0.conv: 0.7000054253472222
Sparsity of body

### Lab2 (e) Global iterative pruning

In [29]:
def global_prune_by_percentage(net, q=70.0):
    """
    Pruning the weight paramters by threshold.
    :param q: pruning percentile. 'q' percent of the least
    significant weight parameters will be pruned.
    """
    # A list to gather all the weights
    flattened_weights = []
    # Find global pruning threshold
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Convert weight to numpy
            weight = layer.weight.data.cpu().numpy()
            # Flatten the weight and append to flattened_weights
            flattened_weights.append(weight.flatten())
    # Concate all weights into a np array
    flattened_weights = np.concatenate(flattened_weights)
    # Find global pruning threshold
    thres = np.percentile(np.abs(flattened_weights), q)
    # Prune the model

    # Apply pruning threshold to all layers
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Convert weight to numpy
            weight = layer.weight.data.cpu().numpy()
            # Generate a binary mask same shape as weight to decide which element to prune
            mask = np.abs(weight) >= thres
            # Convert mask to torch tensor and put on GPU
            mask_tensor = torch.tensor(mask, dtype=torch.float32, device=layer.weight.device)
            # Multiply the weight by mask to perform pruning
            layer.weight.data.mul_(mask_tensor)

In [31]:
net.load_state_dict(torch.load("/content/drive/My Drive/DLHW4/pretrained_model.pt"))
best_acc = 0.0
for epoch in range(20):
    print("\nEpoch: %d" % epoch)
    q = (epoch + 1) * 7

    net.train()
    # Increase model sparsity
    if epoch < 10:
        global_prune_by_percentage(net, q=q)
    if epoch < 9:
        finetune_after_prune(net, trainloader, criterion, optimizer, prune=False)
    else:
        finetune_after_prune(net, trainloader, criterion, optimizer)

    # Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))

    if epoch >= 10:
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving...")
            torch.save(net.state_dict(), "/content/drive/My Drive/DLHW4/net_after_global_iterative_prune.pt")


Epoch: 0


<ipython-input-31-0fae29713af6>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("/content/drive/My Drive/DLHW4/pretrained_model.pt"))


[Step=50]	Loss=0.0473	acc=0.9862	1314.2 examples/second
[Step=100]	Loss=0.0484	acc=0.9847	2610.8 examples/second
[Step=150]	Loss=0.0479	acc=0.9849	2828.4 examples/second
Test Loss=0.3257, Test acc=0.9139

Epoch: 1
[Step=50]	Loss=0.0472	acc=0.9846	1752.6 examples/second
[Step=100]	Loss=0.0479	acc=0.9842	1902.3 examples/second
[Step=150]	Loss=0.0482	acc=0.9846	2562.6 examples/second
Test Loss=0.3255, Test acc=0.9153

Epoch: 2
[Step=50]	Loss=0.0477	acc=0.9849	1425.8 examples/second
[Step=100]	Loss=0.0462	acc=0.9859	2800.7 examples/second
[Step=150]	Loss=0.0473	acc=0.9853	1897.2 examples/second
Test Loss=0.3251, Test acc=0.9139

Epoch: 3
[Step=50]	Loss=0.0484	acc=0.9850	1405.5 examples/second
[Step=100]	Loss=0.0494	acc=0.9846	2464.1 examples/second
[Step=150]	Loss=0.0490	acc=0.9851	2825.1 examples/second
Test Loss=0.3282, Test acc=0.9130

Epoch: 4
[Step=50]	Loss=0.0549	acc=0.9826	1667.0 examples/second
[Step=100]	Loss=0.0534	acc=0.9830	1958.4 examples/second
[Step=150]	Loss=0.0535	acc=0.98

In [32]:
net.load_state_dict(torch.load("/content/drive/My Drive/DLHW4/net_after_global_iterative_prune.pt"))

zeros_sum = 0
total_sum = 0
for name, layer in net.named_modules():
    if (
        isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)
    ) and "id_mapping" not in name:
        # Your code here:
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.data.cpu().numpy()
        # Count number of zeros
        zeros = np.sum(np_weight == 0)
        # Count number of parameters
        total = np_weight.size
        zeros_sum += zeros
        total_sum += total
        print("Sparsity of " + name + ": " + str(zeros / total))
print("Total sparsity of: " + str(zeros_sum / total_sum))
test(net)

<ipython-input-32-e491f61d38f0>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("/content/drive/My Drive/DLHW4/net_after_global_iterative_pru

Sparsity of head_conv.0.conv: 0.24537037037037038
Sparsity of body_op.0.conv1.0.conv: 0.5503472222222222
Sparsity of body_op.0.conv2.0.conv: 0.5277777777777778
Sparsity of body_op.1.conv1.0.conv: 0.5199652777777778
Sparsity of body_op.1.conv2.0.conv: 0.5533854166666666
Sparsity of body_op.2.conv1.0.conv: 0.5164930555555556
Sparsity of body_op.2.conv2.0.conv: 0.5659722222222222
Sparsity of body_op.3.conv1.0.conv: 0.5260416666666666
Sparsity of body_op.3.conv2.0.conv: 0.5831163194444444
Sparsity of body_op.4.conv1.0.conv: 0.6156684027777778
Sparsity of body_op.4.conv2.0.conv: 0.6773003472222222
Sparsity of body_op.5.conv1.0.conv: 0.6115451388888888
Sparsity of body_op.5.conv2.0.conv: 0.7035590277777778
Sparsity of body_op.6.conv1.0.conv: 0.6151801215277778
Sparsity of body_op.6.conv2.0.conv: 0.6510687934027778
Sparsity of body_op.7.conv1.0.conv: 0.6620008680555556
Sparsity of body_op.7.conv2.0.conv: 0.7189670138888888
Sparsity of body_op.8.conv1.0.conv: 0.7479112413194444
Sparsity of bod

### Lab 3 (b) and (c): Fixed-point quantization

In [15]:
# Define quantized model and load weight
Nbits = 4  # Change this value to finish (b) and (c)

net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net = net.to(device)
net.load_state_dict(torch.load("/content/drive/My Drive/DLHW4/pretrained_model.pt"))
test(net)

<ipython-input-15-48327ccf2db4>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("/content/drive/My Drive/DLHW4/pretrained_model.pt"))


Files already downloaded and verified
Test Loss=0.3861, Test accuracy=0.8972


In [16]:
# Quantized model finetuning
finetune(net, epochs=20, batch_size=256, lr=0.002, reg=1e-4)
# Load the model with best accuracy
net.load_state_dict(torch.load("/content/drive/My Drive/DLHW4/quantized_net_after_finetune.pt"))
test(net)


==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=50]	Loss=0.0648	acc=0.9784	1505.9 examples/second
[Step=100]	Loss=0.0657	acc=0.9779	1744.7 examples/second
[Step=150]	Loss=0.0664	acc=0.9777	2427.9 examples/second
Test Loss=0.3332, Test acc=0.9086
Saving to /content/drive/My Drive/DLHW4/quantized_net_after_finetune.pt...

Epoch: 1
[Step=200]	Loss=0.0636	acc=0.9824	1105.5 examples/second
[Step=250]	Loss=0.0654	acc=0.9784	2204.0 examples/second
[Step=300]	Loss=0.0607	acc=0.9799	1775.7 examples/second
[Step=350]	Loss=0.0610	acc=0.9795	2271.5 examples/second
Test Loss=0.3327, Test acc=0.9092
Saving to /content/drive/My Drive/DLHW4/quantized_net_after_finetune.pt...

Epoch: 2
[Step=400]	Loss=0.0523	acc=0.9819	998.2 examples/second
[Step=450]	Loss=0.0610	acc=0.9789	2233.5 examples/second
[Step=500]	Loss=0.0609	acc=0.9792	1686.3 examples/second
[Step=550]	Loss=0.0610	acc=0.9792	2375.2 examples/second
Test Loss=0.3312, Test acc=0.9

<ipython-input-16-54b085f7f189>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("/content/drive/My Drive/DLHW4/quantized_net_after_finetune.p

Files already downloaded and verified
Test Loss=0.3307, Test accuracy=0.9134


### Lab3 (d) Quantize pruned model

In [19]:
# Define quantized model and load weight
Nbits = 2  # Change this value to finish (d)

net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net = net.to(device)
net.load_state_dict(torch.load("/content/drive/My Drive/DLHW4/net_after_global_iterative_prune.pt"))
test(net)

<ipython-input-19-e00dc2658698>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("/content/drive/My Drive/DLHW4/net_after_global_iterative_pru

Files already downloaded and verified
Test Loss=1.0269, Test accuracy=0.6628


In [21]:
# Quantized model finetuning
finetune(net, epochs=20, batch_size=256, lr=0.002, reg=1e-4)

# Load the model with best accuracy
net.load_state_dict(torch.load("/content/drive/My Drive/DLHW4/quantized_net_after_finetune.pt"))
test(net)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=50]	Loss=0.2036	acc=0.9300	1152.0 examples/second
[Step=100]	Loss=0.1975	acc=0.9310	2395.5 examples/second
[Step=150]	Loss=0.1950	acc=0.9316	1735.6 examples/second
Test Loss=0.3717, Test acc=0.8825
Saving to /content/drive/My Drive/DLHW4/quantized_net_after_finetune.pt...

Epoch: 1
[Step=200]	Loss=0.1893	acc=0.9277	1138.3 examples/second
[Step=250]	Loss=0.1966	acc=0.9299	1629.2 examples/second
[Step=300]	Loss=0.1945	acc=0.9310	2367.1 examples/second
[Step=350]	Loss=0.1926	acc=0.9322	1968.8 examples/second
Test Loss=0.3708, Test acc=0.8832
Saving to /content/drive/My Drive/DLHW4/quantized_net_after_finetune.pt...

Epoch: 2
[Step=400]	Loss=0.1918	acc=0.9307	1089.8 examples/second
[Step=450]	Loss=0.1896	acc=0.9337	1694.6 examples/second
[Step=500]	Loss=0.1884	acc=0.9338	2331.7 examples/second
[Step=550]	Loss=0.1859	acc=0.9350	2316.4 examples/second
Test Loss=0.3786, Test acc=0.

<ipython-input-21-40c90cedf075>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("/content/drive/My Drive/DLHW4/quantized_net_after_finetune.p

Files already downloaded and verified
Test Loss=0.3533, Test accuracy=0.8919


### Lab3 (e) Symmetric quantization
#### Implement symmetric quantization in FP_layers.py, and repeat the process in (b)

In [ ]:
# check the performance of symmetric quantization with 6, 5, 4, 3, 2 bits